In [ ]:
using RigidBodyDynamics
using QPControl
using SimpleQP
using Compat.Test
using BenchmarkTools
using StaticArrays

In [ ]:
using OSQP.MathOptInterfaceOSQP
import MathOptInterface
const MOI = MathOptInterface

function defaultoptimizer()
    optimizer = OSQPOptimizer()
    MOI.set!(optimizer, OSQPSettings.Verbose(), false)
    MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-8)
    MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-8)
    MOI.set!(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    optimizer
end

In [ ]:
world = RigidBody{Float64}("world")
mechanism = Mechanism(world, gravity=SVector(0, 0, -9.81))

frame = CartesianFrame3D("core")
inertia = SpatialInertia(frame, SDiagonal(1., 1, 1), SVector(0., 0, 0), 10.0)
body = RigidBody(inertia)
joint = Joint("base_x", Prismatic(SVector(1., 0, 0)))
effort_bounds(joint) .= RigidBodyDynamics.Bounds(-100, 100)
position_bounds(joint) .= RigidBodyDynamics.Bounds(-100, 100)
attach!(mechanism, world, body, joint)

In [ ]:
mpc = MPCController{Void}(mechanism, defaultoptimizer())
stage = addstage!(mpc, 0.001)

objective = @expression 10 * stage.q[1]^2 + 0.01 * stage.v[1]^2 + 1e-7 * stage.u[1]^2
@objective mpc.qpmodel Minimize objective

In [ ]:
state = MechanismState(mechanism)
set_configuration!(state, [1.0])
τ = similar(velocity(state));
benchresult = @benchmark $mpc($τ, 0.0, $state)
@test benchresult.allocs == 0
benchresult

In [ ]:
state = MechanismState(mechanism)
set_configuration!(state, [1.0])
ts, qs, vs = simulate(state, 4.0, mpc; Δt=1e-3)
@time simulate(state, 4.0, mpc; Δt=1e-3);

In [ ]:
#NBSKIP
using Plots
plt = plot(ts, first.(qs))
ylims!(plt, (-1, 1))